## Generate synthesized images from our trained Generator model

In [ ]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import os
import zipfile

In [ ]:
nz = 100
ngf = 64
ndf = 64
nc = 3

In [ ]:
class Generator(nn.Module):
    def __init__(self, ngpu):
        super(Generator, self).__init__()
        self.ngpu = ngpu
        self.main = nn.Sequential(
            nn.ConvTranspose2d( nz, ngf * 8, 4, 1, 0, bias=False),
            nn.BatchNorm2d(ngf * 8),
            nn.ReLU(True),
            # state size. ``(ngf*8) x 4 x 4``
            nn.ConvTranspose2d(ngf * 8, ngf * 4, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 4),
            nn.ReLU(True),
            # state size. ``(ngf*4) x 8 x 8``
            nn.ConvTranspose2d( ngf * 4, ngf * 2, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf * 2),
            nn.ReLU(True),
            # state size. ``(ngf*2) x 16 x 16``
            nn.ConvTranspose2d( ngf * 2, ngf, 4, 2, 1, bias=False),
            nn.BatchNorm2d(ngf),
            nn.ReLU(True),
            # state size. ``(ngf) x 32 x 32``
            nn.ConvTranspose2d( ngf, nc, 4, 2, 1, bias=False),
            nn.Tanh()
            # state size. ``(nc) x 64 x 64``
        )

    def forward(self, input):
        return self.main(input)


### Load the Model

In [ ]:
ngpu = 1

# Inicializar el generador
generator = Generator(ngpu)

# Cargar los pesos preentrenados
PATH = "/content/dcgans_16batch_500epochs_AD_GENERATORpt5.pth"
device = torch.device("cpu")
generator.load_state_dict(torch.load(PATH, map_location=device))

# Poner el generador en modo de evaluación
generator.eval()


Generator(
  (main): Sequential(
    (0): ConvTranspose2d(100, 512, kernel_size=(4, 4), stride=(1, 1), bias=False)
    (1): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): ConvTranspose2d(512, 256, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (4): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): ConvTranspose2d(256, 128, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (7): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): ConvTranspose2d(128, 64, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): ConvTranspose2d(64, 3, kernel_size=(4, 4), stride=(2, 2), padding=(1, 1), bias=False)
    (13): Tanh()
  )
)

### Number of images to generate - Remember to change the name of the output_folder

In [ ]:
# Create a folder to store the generated images
output_folder = "/content/classification_gan_generated_images"
os.makedirs(output_folder, exist_ok=True)

# Number of images to be generated
num_images = 150

# Generar y guardar las imágenes
for i in range(num_images):
    
    nz = 100  
    latent_vector = torch.randn(1, nz, 1, 1)

    # Generating an image using the pretrained generator
    with torch.no_grad():
        generated_image = generator(latent_vector)

    
    generated_image = generated_image / 2 + 0.5

    # Guardar la imagen en la carpeta de salida
    image_path = os.path.join(output_folder, f"generated_image_{i}.png")
    torchvision.utils.save_image(torch.Tensor(generated_image), image_path)

print("Images generated and saved correctly.")

Imágenes generadas y guardadas correctamente.


### Compress the generated images into a ZIP file

In [ ]:
folder_path = '/content/classification_gan_generated_images'  # Path of the folder you want to compress

output_path = '/content/classification_generated_AD_images.zip'  # Path and name of the output compressed file

def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, _, files in os.walk(folder_path):
            for file in files:
                file_path = os.path.join(root, file)
                zipf.write(file_path, arcname=os.path.relpath(file_path, folder_path))

zip_folder(folder_path, output_path)